Ollama 3.2 pour la génération des conversations en json d'aprés les pdf de chaque classe 

#avec pip  PyPDF2

In [10]:
!pip install -q PyPDF2 requests

In [1]:
import requests
import json
import PyPDF2
import os
from pathlib import Path
import time


verif llama


In [2]:
import requests

def check_ollama():
    try:
        response = requests.get(
            "http://localhost:11434/api/tags",
            headers={"Connection": "close"},  # force fermeture de la connexion
            timeout=5                        # timeout de 5s pour éviter de bloquer
        )
        response.raise_for_status()  # force une exception si le code n'est pas 200
        models = response.json().get("models", [])
        print("Ollama actif. Modeles disponibles:")
        for model in models:
            print(f"  - {model['name']}")
        return True
    except requests.exceptions.RequestException as e:
        print("ERREUR: Impossible de contacter Ollama.")
        print("Détails:", e)
        print("Assure-toi que Ollama tourne et que le firewall autorise Python.")
        return False

if check_ollama():
    print("\nOllama prêt")
else:
    print("\nDemarre Ollama avant de continuer")


Ollama actif. Modeles disponibles:
  - mistral:7b-instruct
  - llama3.2:latest

Ollama prêt



Fonction extraction PDF


In [3]:
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

print("Fonction extraction PDF prete")

Fonction extraction PDF prete



Fonction génération avec Ollama


In [4]:
# def generate_with_ollama(prompt, model="llama3.2"):
#     url = "http://localhost:11434/api/generate"
    
#     payload = {
#         "model": model,
#         "prompt": prompt,
#         "stream": False,
#         "options": {
#             "temperature": 0.7,
#             "top_p": 0.9,
#             "num_predict": 4000
#         }
#     }
    
#     try:
#         response = requests.post(url, json=payload, timeout=300)
#         if response.status_code == 200:
#             return response.json()["response"]
#         else:
#             print(f"Erreur API: {response.status_code}")
#             return None
#     except Exception as e:
#         print(f"Erreur requete: {e}")
#         return None

# print("Fonction generation Ollama prete")


#------------------------------------------------------------

# def generate_with_ollama(prompt, model="mistral:7b-instruct"):
#     """Génération optimisée avec Mistral"""
#     url = "http://localhost:11434/api/generate"
    
#     payload = {
#         "model": model,
#         "prompt": prompt,
#         "stream": False,
#         "options": {
#             "temperature": 0.2,      # ⬇️ Plus strict
#             "top_p": 0.85,           # ⬇️ Moins de variabilité  
#             "top_k": 20,             # 🆕 Limite les choix
#             "num_predict": 6000,     # ⬆️ Plus long
#             "repeat_penalty": 1.2,   # 🆕 Évite répétitions
#             "stop": ["</s>", "###"]  # 🆕 Stop tokens
#         }
#     }
    
#     try:
#         response = requests.post(url, json=payload, timeout=600)  # ⬆️ 10 min timeout
#         if response.status_code == 200:
#             return response.json()["response"]
#         else:
#             print(f"❌ Erreur API: {response.status_code}")
#             return None
#     except Exception as e:
#         print(f"❌ Erreur: {e}")
#         return None

# print("✅ Fonction Mistral optimisée prête")

def generate_with_ollama(prompt, model="mistral:7b-instruct"):
    """Génération avec Mistral - paramètres optimisés"""
    url = "http://localhost:11434/api/generate"
    
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False,
        "options": {
            "temperature": 0.3,      # Plus strict pour suivre instructions
            "top_p": 0.9,
            "top_k": 40,
            "num_predict": 8000,     # Plus de tokens
            "repeat_penalty": 1.1
        }
    }
    
    try:
        response = requests.post(url, json=payload, timeout=900)
        if response.status_code == 200:
            return response.json()["response"]
        else:
            print(f"Erreur API: {response.status_code}")
            return None
    except Exception as e:
        print(f"Erreur: {e}")
        return None

print("✅ Fonction Mistral prête")

✅ Fonction Mistral prête



Fonction création conversations médicales


In [5]:
# # def create_medical_conversations(protocol_text, disease_class, num_conversations=150):
# #     conversations_list = []
# #     batches = num_conversations // 5
# #     successful_batches = 0
    
# #     for batch in range(batches):
# #         print(f"  Batch {batch+1}/{batches}...", end=" ")
        
# #         prompt = f"""Tu es un generateur de dialogues medicaux specialise en dermatologie.

# # PROTOCOLE MEDICAL OFFICIEL pour {disease_class}:
# # {protocol_text[:7000]}

# # TACHE:
# # Genere 5 conversations COMPLETES et PROFESSIONNELLES entre dermatologue et patient.
# # Chaque conversation DOIT suivre ce flux OBLIGATOIRE:

# # 1. System: Diagnostic preliminaire
# # 2. Dermatologue: Chronologie (depuis quand, apparition brutale/progressive)
# # 3. Patient: Reponse realiste
# # 4. Dermatologue: Duree lesions individuelles (< ou > 24h)
# # 5. Patient: Reponse
# # 6. Dermatologue: Intensite demangeaisons
# # 7. Patient: Reponse
# # 8. Dermatologue: Medicaments recents (24-48h)
# # 9. Patient: Reponse
# # 10. Dermatologue: SIGNES URGENCE (dyspnee, angio-oedeme)
# # 11. Patient: Reponse
# # 12. Dermatologue: Fievre/infection recente
# # 13. Patient: Reponse
# # 14. Dermatologue: DIAGNOSTIC FINAL + traitement recommande

# # VARIATIONS OBLIGATOIRES entre conversations:
# # - Conv 1-2: Urticaire aigue medicamenteuse (AINS, aspirine, antibiotiques)
# # - Conv 3: Urticaire post-infectieuse (rhume, gastro-enterite)
# # - Conv 4: Urticaire alimentaire (fruits de mer, arachides, oeufs)
# # - Conv 5: Urticaire avec signes urgence (angio-oedeme, dyspnee legere)

# # REGLES STRICTES:
# # - Vocabulaire medical technique UNIQUEMENT
# # - AUCUNE invention (pas de "sauge", "poison", "cigarette")
# # - TOUJOURS terminer par diagnostic + traitement
# # - Questions selon protocole UNIQUEMENT
# # - AUCUNE conversation incomplete
# # IMPORTANTE: Tu DOIS retourner un JSON strict, valide, et **aucune conversation incomplète**. Si tu ne peux pas générer 14 étapes, retourne un message d'erreur.


# # FORMAT JSON:
# # [
# #   {{
# #     "image_id": "id_001",
# #     "disease_class": "{disease_class}",
# #     "conversations": [
# #       {{"from": "system", "value": "Diagnostic preliminaire: {disease_class}"}},
# #       {{"from": "assistant", "value": "J'observe sur l'image des lesions cutanees evocatrices d'urticaire. Depuis combien de temps exactement ces lesions sont-elles presentes?"}},
# #       {{"from": "user", "value": "Depuis hier soir, environ 18 heures."}},
# #       {{"from": "assistant", "value": "Ces lesions sont-elles apparues de maniere brutale en quelques minutes a heures, ou de maniere progressive?"}},
# #       {{"from": "user", "value": "Tres rapidement, en moins de 2 heures."}},
# #       {{"from": "assistant", "value": "Quelle est la duree de persistance de chaque lesion individuelle? Restent-elles fixes plus de 24 heures ou disparaissent-elles et reapparaissent ailleurs?"}},
# #       {{"from": "user", "value": "Elles disparaissent en quelques heures et reapparaissent a d'autres endroits."}},
# #       {{"from": "assistant", "value": "Ressentez-vous des demangeaisons? Si oui, quelle est leur intensite?"}},
# #       {{"from": "user", "value": "Oui, des demangeaisons tres intenses."}},
# #       {{"from": "assistant", "value": "Avez-vous pris un nouveau medicament dans les 24 a 48 dernieres heures? Antibiotiques, anti-inflammatoires non steroidiens, aspirine?"}},
# #       {{"from": "user", "value": "J'ai pris de l'ibuprofene hier matin pour un mal de tete."}},
# #       {{"from": "assistant", "value": "Les AINS comme l'ibuprofene peuvent declencher ou aggraver une urticaire. Presentez-vous actuellement des difficultes respiratoires, un gonflement du visage, des levres ou de la gorge?"}},
# #       {{"from": "user", "value": "Non, aucun gonflement ni difficulte respiratoire."}},
# #       {{"from": "assistant", "value": "Avez-vous presente recemment une fievre ou des symptomes infectieux? Infection respiratoire, gastro-intestinale, otite?"}},
# #       {{"from": "user", "value": "J'ai eu un rhume il y a 3 jours mais plus de fievre maintenant."}},
# #       {{"from": "assistant", "value": "Diagnostic retenu: urticaire aigue probable d'origine medicamenteuse (AINS) et post-infectieuse. Traitement recommande: antihistaminiques H1 (cetirizine 10 mg par jour), eviction de l'ibuprofene, surveillance pendant 48 heures. Consultation urgente si apparition de signes d'anaphylaxie."}}
# #     ]
# #   }}
# # ]

# # GENERE MAINTENANT LES 5 CONVERSATIONS. JSON UNIQUEMENT:"""

# #         response_text = generate_with_ollama(prompt)
        
# #         if not response_text:
# #             print("Pas de reponse")
# #             continue
            
# #         try:
# #             cleaned = response_text.strip()
# #             if "```json" in cleaned:
# #                 cleaned = cleaned.split("```json")[1].split("```")[0]
# #             elif "```" in cleaned:
# #                 cleaned = cleaned.split("```")[1].split("```")[0]
            
# #             json_start = cleaned.find('[')
# #             json_end = cleaned.rfind(']') + 1
            
# #             if json_start == -1 or json_end <= json_start:
# #                 print("Pas de JSON")
# #                 continue
            
# #             json_str = cleaned[json_start:json_end]
# #             json_str = json_str.replace('\n', ' ').replace('\r', ' ')
# #             json_str = ' '.join(json_str.split())
            
# #             try:
# #                 batch_data = json.loads(json_str)
# #             except json.JSONDecodeError:
# #                 parts = []
# #                 depth = 0
# #                 current = ""
# #                 for char in json_str:
# #                     current += char
# #                     if char == '{':
# #                         depth += 1
# #                     elif char == '}':
# #                         depth -= 1
# #                         if depth == 0 and current.strip().startswith('{'):
# #                             parts.append(current.strip())
# #                             current = ""
# #                 if parts:
# #                     batch_data = []
# #                     for part in parts:
# #                         try:
# #                             obj = json.loads(part)
# #                             if isinstance(obj, dict) and "conversations" in obj:
# #                                 if len(obj["conversations"]) >= 14:
# #                                     batch_data.append(obj)
# #                         except:
# #                             continue
# #                 else:
# #                     print("JSON invalide")
# #                     continue
            
# #             if not isinstance(batch_data, list):
# #                 batch_data = [batch_data]
            
# #             added = 0
# #             for conv in batch_data:
# #                 if isinstance(conv, dict) and "conversations" in conv:
# #                     if len(conv["conversations"]) >= 14:
# #                         base_name = disease_class.lower()
# #                         for old, new in [(' ', '_'), ('é', 'e'), ('è', 'e'), ('ê', 'e'), ('î', 'i'), ('ô', 'o')]:
# #                             base_name = base_name.replace(old, new)
                        
# #                         conv['image_id'] = f"{base_name}_{len(conversations_list) + 1:03d}"
# #                         conv['disease_class'] = disease_class
# #                         conversations_list.append(conv)
# #                         added += 1
            
# #             if added > 0:
# #                 successful_batches += 1
# #                 print(f"{added} OK")
# #             else:
# #                 print("Trop courtes")
                
# #         except Exception as e:
# #             print(f"Erreur")
        
# #         time.sleep(1)
    
# #     return conversations_list

# # print("Fonction AMELIOREE prete")





# #---------------------------------------------------------------------------------2eme essai---------------------------------------------

# def create_medical_conversations(protocol_text, disease_class, num_conversations=150):
#     conversations_list = []
#     batches = num_conversations // 3
    
#     for batch in range(batches):
#         print(f"  Batch {batch+1}/{batches}...", end=" ")
        
#         # 🔥 PROMPT ULTRA-SIMPLIFIÉ
#         prompt = f"""Tu dois générer 3 conversations médicales en JSON.

# EXEMPLE EXACT À SUIVRE:
# [
#   {{
#     "image_id": "urticaria_001",
#     "disease_class": "Urticaria Hives",
#     "conversations": [
#       {{"from": "system", "value": "Diagnostic préliminaire: Urticaria Hives"}},
#       {{"from": "assistant", "value": "Depuis combien de temps ces lésions?"}},
#       {{"from": "user", "value": "Depuis hier 18h."}},
#       {{"from": "assistant", "value": "Apparition brutale ou progressive?"}},
#       {{"from": "user", "value": "Très rapide, moins de 2h."}},
#       {{"from": "assistant", "value": "Durée de chaque lésion? Plus ou moins de 24h?"}},
#       {{"from": "user", "value": "Elles disparaissent en quelques heures."}},
#       {{"from": "assistant", "value": "Démangeaisons? Intensité?"}},
#       {{"from": "user", "value": "Oui, très intenses."}},
#       {{"from": "assistant", "value": "Nouveau médicament dans les 48h?"}},
#       {{"from": "user", "value": "Ibuprofène hier matin."}},
#       {{"from": "assistant", "value": "L'ibuprofène peut causer l'urticaire. Difficultés respiratoires? Gonflement visage?"}},
#       {{"from": "user", "value": "Non, aucun gonflement."}},
#       {{"from": "assistant", "value": "Fièvre ou infection récente?"}},
#       {{"from": "user", "value": "Rhume il y a 3 jours."}},
#       {{"from": "assistant", "value": "Diagnostic: urticaire aiguë médicamenteuse et post-infectieuse. Traitement: cétirizine 10mg/jour, éviction ibuprofène, surveillance 48h."}}
#     ]
#   }}
# ]

# PROTOCOLE {disease_class}:
# {protocol_text[:4000]}

# Génère 3 conversations similaires en JSON pur (commence par [ et termine par ]):"""

#         response_text = generate_with_ollama(prompt, model="mistral:7b-instruct")
        
#         # 🔍 DEBUG: AFFICHE LA RÉPONSE BRUTE
#         if response_text:
#             print(f"\n📝 RÉPONSE BRUTE (100 premiers caractères):")
#             print(response_text[:200])
#             print("...")
#         else:
#             print("❌ Pas de réponse")
#             continue
        
#         # 🔧 PARSING JSON
#         try:
#             cleaned = response_text.strip()
            
#             # Enlève markdown
#             if "```json" in cleaned:
#                 cleaned = cleaned.split("```json")[1].split("```")[0]
#             elif "```" in cleaned:
#                 cleaned = cleaned.split("```")[1].split("```")[0]
            
#             # Trouve le JSON
#             json_start = cleaned.find('[')
#             json_end = cleaned.rfind(']') + 1
            
#             if json_start == -1 or json_end <= json_start:
#                 print("❌ Pas de JSON valide trouvé")
#                 print(f"Début: {cleaned[:100]}")
#                 continue
            
#             json_str = cleaned[json_start:json_end]
            
#             print(f"🔍 JSON extrait (100 premiers caractères): {json_str[:100]}")
            
#             # Parse
#             try:
#                 batch_data = json.loads(json_str)
#                 print(f"✅ JSON parsé avec succès")
#             except json.JSONDecodeError as e:
#                 print(f"❌ Erreur JSON: {e}")
#                 print(f"Position erreur: {e.pos}")
#                 print(f"Extrait problématique: {json_str[max(0,e.pos-50):e.pos+50]}")
#                 continue
            
#             if not isinstance(batch_data, list):
#                 batch_data = [batch_data]
            
#             # Validation
#             added = 0
#             for conv in batch_data:
#                 if isinstance(conv, dict) and "conversations" in conv:
#                     n_exchanges = len(conv["conversations"])
#                     print(f"  Conv trouvée: {n_exchanges} échanges")
                    
#                     if n_exchanges >= 16:
#                         last_msg = conv["conversations"][-1]["value"].lower()
#                         has_diagnosis = "diagnostic" in last_msg
#                         has_treatment = "traitement" in last_msg or "cetirizine" in last_msg
                        
#                         if has_diagnosis and has_treatment:
#                             base_name = disease_class.lower().replace(' ', '_').replace('é', 'e')
#                             conv['image_id'] = f"{base_name}_{len(conversations_list) + 1:03d}"
#                             conv['disease_class'] = disease_class
#                             conversations_list.append(conv)
#                             added += 1
#                             print(f"    ✅ Acceptée")
#                         else:
#                             print(f"    ❌ Rejetée (pas de diag/trait)")
#                     else:
#                         print(f"    ❌ Rejetée (trop courte)")
            
#             if added > 0:
#                 print(f"✅ {added} conversations ajoutées")
#             else:
#                 print("❌ Aucune conversation valide")
                
#         except Exception as e:
#             print(f"❌ Erreur parsing: {e}")
        
#         time.sleep(2)
    
#     return conversations_list

# print("✅ Fonction DEBUG prête")

def create_medical_conversations(protocol_text, disease_class, num_conversations=150):
    """Génère des conversations médicales de haute qualité"""
    conversations_list = []
    batches = num_conversations // 2  # 2 conversations par batch (plus fiable)
    
    for batch in range(batches):
        print(f"  Batch {batch+1}/{batches}...", end=" ", flush=True)
        
        # PROMPT ULTRA-OPTIMISÉ
        prompt = f"""Génère EXACTEMENT 2 conversations médicales dermatologiques en JSON.

EXEMPLE À SUIVRE (16 ÉCHANGES):
[
{{
  "image_id": "urticaria_001",
  "disease_class": "Urticaria Hives",
  "conversations": [
    {{"from": "system", "value": "Diagnostic préliminaire: Urticaria Hives"}},
    {{"from": "assistant", "value": "Depuis combien de temps exactement ces lésions cutanées sont-elles présentes?"}},
    {{"from": "user", "value": "Depuis hier soir, environ 18 heures."}},
    {{"from": "assistant", "value": "Ces lésions sont-elles apparues de manière brutale en quelques minutes ou heures, ou de manière progressive sur plusieurs jours?"}},
    {{"from": "user", "value": "Très rapidement, en moins de 2 heures."}},
    {{"from": "assistant", "value": "Quelle est la durée de persistance de chaque lésion individuelle? Restent-elles fixes plus de 24 heures ou disparaissent-elles et réapparaissent ailleurs?"}},
    {{"from": "user", "value": "Elles disparaissent en quelques heures et réapparaissent à d'autres endroits."}},
    {{"from": "assistant", "value": "Ressentez-vous des démangeaisons? Si oui, évaluez leur intensité."}},
    {{"from": "user", "value": "Oui, des démangeaisons très intenses."}},
    {{"from": "assistant", "value": "Avez-vous pris un nouveau médicament dans les 24 à 48 dernières heures? Antibiotiques, anti-inflammatoires, aspirine?"}},
    {{"from": "user", "value": "J'ai pris de l'ibuprofène hier matin pour un mal de tête."}},
    {{"from": "assistant", "value": "Les anti-inflammatoires non stéroïdiens comme l'ibuprofène peuvent déclencher ou aggraver une urticaire. Présentez-vous actuellement des difficultés respiratoires, un gonflement du visage, des lèvres ou de la gorge?"}},
    {{"from": "user", "value": "Non, aucun gonflement ni difficulté respiratoire."}},
    {{"from": "assistant", "value": "Avez-vous présenté récemment une fièvre ou des symptômes infectieux? Infection respiratoire, gastro-intestinale, otite?"}},
    {{"from": "user", "value": "J'ai eu un rhume il y a 3 jours mais plus de fièvre maintenant."}},
    {{"from": "assistant", "value": "Diagnostic retenu: urticaire aiguë probable d'origine médicamenteuse et/ou post-infectieuse. Traitement recommandé: antihistaminiques H1 (cétirizine 10 mg par jour), éviction de l'ibuprofène, surveillance pendant 48 heures. Consultation urgente si apparition de signes d'anaphylaxie."}}
  ]
}}
]

PROTOCOLE: {disease_class}
{protocol_text[:4500]}

RÈGLES STRICTES:
- Vocabulaire médical technique UNIQUEMENT
- 16 échanges par conversation (8 questions + 8 réponses)
- Diagnostic final COMPLET avec traitement et posologie
- INTERDITS: sauge, poison, cigarette, auto-infection
- Réponses patients réalistes et précises

SCÉNARIOS (1 par conversation):
- Conv 1: Urticaire médicamenteuse AINS
- Conv 2: Urticaire post-infectieuse virale

Génère 2 conversations en JSON (commence par [ et termine par ]):"""

        response_text = generate_with_ollama(prompt, model="mistral:7b-instruct")
        
        if not response_text:
            print("❌ Pas réponse")
            continue
        
        # PARSING JSON ROBUSTE
        try:
            cleaned = response_text.strip()
            
            # Enlève markdown
            if "```json" in cleaned:
                cleaned = cleaned.split("```json")[1].split("```")[0]
            elif "```" in cleaned:
                cleaned = cleaned.split("```")[1].split("```")[0]
            
            # Trouve JSON
            json_start = cleaned.find('[')
            json_end = cleaned.rfind(']') + 1
            
            if json_start == -1 or json_end <= json_start:
                print("❌ JSON invalide")
                continue
            
            json_str = cleaned[json_start:json_end]
            
            # Parse
            try:
                batch_data = json.loads(json_str)
            except json.JSONDecodeError:
                print("❌ Parse échec")
                continue
            
            if not isinstance(batch_data, list):
                batch_data = [batch_data]
            
            # VALIDATION AVEC CRITÈRES ASSOUPLIS
            added = 0
            for conv in batch_data:
                if isinstance(conv, dict) and "conversations" in conv:
                    n_exchanges = len(conv["conversations"])
                    
                    # ACCEPTE SI >= 14 ÉCHANGES (au lieu de 16 strict)
                    if n_exchanges >= 14:
                        last_msg = conv["conversations"][-1]["value"].lower()
                        
                        # Critères qualité
                        has_diagnosis = "diagnostic" in last_msg or "retenu" in last_msg
                        has_treatment = "traitement" in last_msg or "cetirizine" in last_msg or "antihistaminique" in last_msg
                        
                        # Mots interdits
                        full_conv = ' '.join([msg['value'].lower() for msg in conv['conversations']])
                        forbidden = ["sauge", "poison", "cigarette", "auto-infection"]
                        has_forbidden = any(word in full_conv for word in forbidden)
                        
                        # ACCEPTE SI: diag + trait + pas de mots interdits
                        if has_diagnosis and has_treatment and not has_forbidden:
                            base_name = disease_class.lower().replace(' ', '_').replace('é', 'e').replace('è', 'e')
                            conv['image_id'] = f"{base_name}_{len(conversations_list) + 1:03d}"
                            conv['disease_class'] = disease_class
                            conversations_list.append(conv)
                            added += 1
            
            if added > 0:
                print(f"✅ {added}")
            else:
                print("❌ Rejeté")
                
        except Exception as e:
            print(f"❌ Erreur")
        
        time.sleep(3)  # Pause entre batches
    
    return conversations_list

print("✅ Fonction conversations prête")


✅ Fonction conversations prête


In [8]:
from pathlib import Path

docs_folder = Path("docs")
test_class = "Urticaria Hives"
test_pdf = "Urticaria Hives.pdf"
test_path = docs_folder / test_pdf

if test_path.exists():
    protocol = extract_text_from_pdf(test_path)
    print(f"✓ Protocole: {len(protocol)} caractères\n")
    
    # GÉNÈRE 3 CONVERSATIONS (1 batch)
    test_convs = create_medical_conversations(protocol, test_class, num_conversations=3)
    
    print(f"\n📊 RÉSULTAT FINAL: {len(test_convs)}/3 conversations valides")
else:
    print(f"❌ {test_pdf} introuvable")

✓ Protocole: 8852 caractères

  Batch 1/1... ✅ 2

📊 RÉSULTAT FINAL: 2/3 conversations valides


tets--------

In [9]:
from pathlib import Path

docs_folder = Path("docs")
output_folder = Path("conversations_data")
output_folder.mkdir(exist_ok=True)

classes_config = {
    "Urticaria Hives": "Urticaria Hives.pdf",
    "Eczema": "Eczema.pdf",
    "Atopic Dermatitis": "Atopic Dermatitis.pdf",
    "drug reactions": "drug reactions.pdf",
    "Dermatophytosis": "Dermatophytosis.pdf",
    "Gale": "Gale.pdf",
    "Impétigo": "Impétigo.pdf",
    "Maladie cutanée auto-immune": "Maladie cutanée auto-immune.pdf"
}

NUM_CONVERSATIONS_PER_CLASS = 150

print(f"🚀 GÉNÉRATION DE {NUM_CONVERSATIONS_PER_CLASS} CONVERSATIONS PAR CLASSE")
print(f"Total: {len(classes_config)} classes")
print("="*70)

for disease_class, pdf_filename in classes_config.items():
    pdf_path = docs_folder / pdf_filename
    
    if not pdf_path.exists():
        print(f"\n⚠️ {pdf_filename} introuvable - PASSÉ")
        continue
    
    print(f"\n📋 [{disease_class}]")
    print(f"PDF: {pdf_filename}")
    
    protocol_text = extract_text_from_pdf(pdf_path)
    print(f"Protocole: {len(protocol_text)} caractères")
    
    conversations = create_medical_conversations(
        protocol_text, 
        disease_class, 
        NUM_CONVERSATIONS_PER_CLASS
    )
    
    if len(conversations) >= 100:  # Au moins 66% de réussite
        safe_filename = disease_class.lower().replace(' ', '_').replace('é', 'e').replace('è', 'e').replace('î', 'i')
        output_filename = f"{safe_filename}_conversations.json"
        output_path = output_folder / output_filename
        
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(conversations, f, ensure_ascii=False, indent=2)
        
        print(f"✅ SAUVEGARDÉ: {len(conversations)}/{NUM_CONVERSATIONS_PER_CLASS} conversations → {output_filename}")
    else:
        print(f"⚠️ Seulement {len(conversations)}/{NUM_CONVERSATIONS_PER_CLASS} conversations - Pas sauvegardé")
    
    print("-"*70)

print(f"\n{'='*70}")
print("✅ GÉNÉRATION TERMINÉE")
print(f"Dossier: {output_folder.absolute()}")

🚀 GÉNÉRATION DE 150 CONVERSATIONS PAR CLASSE
Total: 8 classes

📋 [Urticaria Hives]
PDF: Urticaria Hives.pdf
Protocole: 8852 caractères
  Batch 1/75... ❌ Rejeté
  Batch 2/75... ✅ 2
  Batch 3/75... ✅ 2
  Batch 4/75... ✅ 2
  Batch 5/75... ✅ 2
  Batch 6/75... ✅ 2
  Batch 7/75... ✅ 2
  Batch 8/75... ✅ 2
  Batch 9/75... ✅ 2
  Batch 10/75... ✅ 2
  Batch 11/75... ✅ 2
  Batch 12/75... ❌ Rejeté
  Batch 13/75... ✅ 2
  Batch 14/75... ✅ 2
  Batch 15/75... ✅ 2
  Batch 16/75... ✅ 2
  Batch 17/75... ✅ 2
  Batch 18/75... ✅ 2
  Batch 19/75... ✅ 2
  Batch 20/75... ✅ 2
  Batch 21/75... ✅ 2
  Batch 22/75... ✅ 2
  Batch 23/75... ✅ 2
  Batch 24/75... ✅ 2
  Batch 25/75... ✅ 1
  Batch 26/75... ✅ 2
  Batch 27/75... ✅ 2
  Batch 28/75... ✅ 2
  Batch 29/75... ❌ Rejeté
  Batch 30/75... 

KeyboardInterrupt: 


Configuration et génération


In [7]:
from pathlib import Path
import json
import PyPDF2
import time
import requests

# ============================================
# CONFIGURATION
# ============================================
docs_folder = Path("docs")
output_folder = Path("conversations_data")
output_folder.mkdir(exist_ok=True)

classes_config = {
    "Urticaria Hives": "Urticaria Hives.pdf",
    "Eczema": "Eczema.pdf",
    "Atopic Dermatitis": "Atopic Dermatitis.pdf",
    "drug reactions": "drug reactions.pdf",
    "Dermatophytosis": "Dermatophytosis.pdf",
    "Gale": "Gale.pdf",
    "Impétigo": "Impétigo.pdf",
    "Maladie cutanée auto-immune": "Maladie cutanée auto-immune.pdf"
}

NUM_CONVERSATIONS_PER_CLASS = 150

# ============================================
# 1. VÉRIFICATION DE L'ÉTAT ACTUEL
# ============================================
print("="*70)
print("🔍 ÉTAT ACTUEL DES FICHIERS")
print("="*70)

progress_report = {}

for disease_class in classes_config.keys():
    safe_filename = disease_class.lower().replace(' ', '_').replace('é', 'e').replace('è', 'e').replace('î', 'i')
    output_filename = f"{safe_filename}_conversations.json"
    output_path = output_folder / output_filename
    
    if output_path.exists():
        try:
            with open(output_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                count = len(data)
                progress_report[disease_class] = {
                    'count': count,
                    'remaining': NUM_CONVERSATIONS_PER_CLASS - count,
                    'complete': count >= NUM_CONVERSATIONS_PER_CLASS,
                    'file': output_filename
                }
                status = "✅ COMPLET" if count >= NUM_CONVERSATIONS_PER_CLASS else f"⚠️ INCOMPLET ({count}/{NUM_CONVERSATIONS_PER_CLASS})"
                print(f"{status} - {disease_class}")
        except:
            progress_report[disease_class] = {
                'count': 0,
                'remaining': NUM_CONVERSATIONS_PER_CLASS,
                'complete': False,
                'file': output_filename
            }
            print(f"❌ ERREUR LECTURE - {disease_class}")
    else:
        progress_report[disease_class] = {
            'count': 0,
            'remaining': NUM_CONVERSATIONS_PER_CLASS,
            'complete': False,
            'file': output_filename
        }
        print(f"📂 PAS DE FICHIER - {disease_class}")

print("="*70)
print("\n🚀 REPRISE DE LA GÉNÉRATION\n")

# ============================================
# 2. GÉNÉRATION DES CONVERSATIONS MANQUANTES
# ============================================
for disease_class, pdf_filename in classes_config.items():
    info = progress_report[disease_class]
    
    print(f"\n{'='*70}")
    print(f"[{disease_class}]")
    print(f"{'='*70}")
    
    # Si déjà complet, on passe
    if info['complete']:
        print(f"✅ Déjà {info['count']} conversations - IGNORÉ")
        continue
    
    # Charge les conversations existantes
    safe_filename = disease_class.lower().replace(' ', '_').replace('é', 'e').replace('è', 'e').replace('î', 'i')
    output_filename = f"{safe_filename}_conversations.json"
    output_path = output_folder / output_filename
    
    existing_conversations = []
    if output_path.exists():
        try:
            with open(output_path, 'r', encoding='utf-8') as f:
                existing_conversations = json.load(f)
            print(f"📂 {len(existing_conversations)} conversations existantes trouvées")
        except:
            print("⚠️ Fichier existant illisible - recommence à 0")
    else:
        print(f"📂 Aucune conversation existante - commence à 0")
    
    # Vérifie si le PDF existe
    pdf_path = docs_folder / pdf_filename
    if not pdf_path.exists():
        print(f"❌ PDF introuvable: {pdf_filename}")
        continue
    
    # Extraction du protocole
    print(f"📄 Extraction du PDF: {pdf_filename}")
    protocol_text = extract_text_from_pdf(pdf_path)
    print(f"✅ {len(protocol_text)} caractères extraits")
    
    # Génère les conversations manquantes
    remaining = info['remaining']
    print(f"\n🔄 Génération de {remaining} conversations supplémentaires...")
    print(f"   (objectif: {NUM_CONVERSATIONS_PER_CLASS} total)")
    
    new_conversations = create_medical_conversations(
        protocol_text, 
        disease_class, 
        remaining
    )
    
    print(f"\n📊 Résultat: {len(new_conversations)} nouvelles conversations générées")
    
    # Combine et sauvegarde
    if len(new_conversations) > 0:
        all_conversations = existing_conversations + new_conversations
        
        # Renumérotation des IDs
        base_name = safe_filename
        for idx, conv in enumerate(all_conversations, 1):
            conv['image_id'] = f"{base_name}_{idx:03d}"
            conv['disease_class'] = disease_class
        
        # Sauvegarde
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(all_conversations, f, ensure_ascii=False, indent=2)
        
        print(f"\n💾 SAUVEGARDE RÉUSSIE:")
        print(f"   📁 Fichier: {output_filename}")
        print(f"   📊 Total: {len(all_conversations)} conversations")
        print(f"   ➕ Anciennes: {len(existing_conversations)}")
        print(f"   🆕 Nouvelles: {len(new_conversations)}")
        
        # Indication de progression
        percentage = (len(all_conversations) / NUM_CONVERSATIONS_PER_CLASS) * 100
        print(f"   📈 Progression: {len(all_conversations)}/{NUM_CONVERSATIONS_PER_CLASS} ({percentage:.1f}%)")
        
        if len(all_conversations) >= NUM_CONVERSATIONS_PER_CLASS:
            print(f"   ✅ CLASSE COMPLÈTE!")
        else:
            print(f"   ⚠️ Encore {NUM_CONVERSATIONS_PER_CLASS - len(all_conversations)} conversations à générer")
    else:
        print(f"❌ Aucune nouvelle conversation générée")
    
    print("-"*70)

# ============================================
# 3. RAPPORT FINAL
# ============================================
print("\n" + "="*70)
print("📊 RAPPORT FINAL")
print("="*70)

total_generated = 0
total_expected = NUM_CONVERSATIONS_PER_CLASS * len(classes_config)

for disease_class in classes_config.keys():
    safe_filename = disease_class.lower().replace(' ', '_').replace('é', 'e').replace('è', 'e').replace('î', 'i')
    output_filename = f"{safe_filename}_conversations.json"
    output_path = output_folder / output_filename
    
    if output_path.exists():
        try:
            with open(output_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                count = len(data)
                total_generated += count
                percentage = (count / NUM_CONVERSATIONS_PER_CLASS) * 100
                
                if count >= NUM_CONVERSATIONS_PER_CLASS:
                    print(f"✅ {disease_class}: {count}/{NUM_CONVERSATIONS_PER_CLASS} ({percentage:.1f}%)")
                else:
                    print(f"⚠️ {disease_class}: {count}/{NUM_CONVERSATIONS_PER_CLASS} ({percentage:.1f}%)")
        except:
            print(f"❌ {disease_class}: Erreur lecture fichier")
    else:
        print(f"❌ {disease_class}: Pas de fichier généré")

print("="*70)
print(f"🎯 TOTAL: {total_generated}/{total_expected} conversations")
print(f"📁 Dossier de sortie: {output_folder.absolute()}")
print("="*70)

# Liste les fichiers créés
print("\n📂 FICHIERS CRÉÉS:")
for file in sorted(output_folder.glob("*.json")):
    size_kb = file.stat().st_size / 1024
    print(f"   - {file.name} ({size_kb:.1f} KB)")

print("\n✅ GÉNÉRATION TERMINÉE")

🔍 ÉTAT ACTUEL DES FICHIERS
✅ COMPLET - Maladie cutanée auto-immune

🚀 REPRISE DE LA GÉNÉRATION


[Maladie cutanée auto-immune]
✅ Déjà 150 conversations - IGNORÉ

📊 RAPPORT FINAL
✅ Maladie cutanée auto-immune: 150/150 (100.0%)
🎯 TOTAL: 150/150 conversations
📁 Dossier de sortie: C:\Users\user\Desktop\projetIA\conversations_data

📂 FICHIERS CRÉÉS:
   - atopic_dermatitis_conversations.json (361.5 KB)
   - dermatophytosis_conversations.json (363.5 KB)
   - drug_reactions_conversations.json (353.6 KB)
   - eczema_conversations.json (388.6 KB)
   - gale_conversations.json (356.8 KB)
   - impetigo_conversations.json (350.9 KB)
   - maladie_cutanee_auto-immune_conversations.json (389.8 KB)
   - urticaria_hives_conversations.json (384.0 KB)

✅ GÉNÉRATION TERMINÉE



Verif finale


In [7]:
from pathlib import Path
import json
import PyPDF2
import time
import requests

docs_folder = Path("docs")
output_folder = Path("conversations_data")
output_folder.mkdir(exist_ok=True)

classes_config = {
    "Urticaria Hives": "Urticaria Hives.pdf",
    "Eczema": "Eczema.pdf",
    "Atopic Dermatitis": "Atopic Dermatitis.pdf",
    "drug reactions": "drug reactions.pdf",
    "Dermatophytosis": "Dermatophytosis.pdf",
    "Gale": "Gale.pdf",
    "Impétigo": "Impétigo.pdf",
    "Maladie cutanée auto-immune": "Maladie cutanée auto-immune.pdf"
}

NUM_CONVERSATIONS_PER_CLASS = 150

# ============================================
# FONCTION POUR VÉRIFIER L'ÉTAT D'AVANCEMENT
# ============================================
def get_progress_status():
    """Vérifie quelles classes sont déjà complètes"""
    progress = {}
    
    for disease_class in classes_config.keys():
        safe_filename = disease_class.lower().replace(' ', '_').replace('é', 'e').replace('è', 'e').replace('î', 'i')
        output_filename = f"{safe_filename}_conversations.json"
        output_path = output_folder / output_filename
        
        if output_path.exists():
            try:
                with open(output_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    progress[disease_class] = {
                        'completed': len(data),
                        'file': output_filename,
                        'needs_more': len(data) < NUM_CONVERSATIONS_PER_CLASS
                    }
            except:
                progress[disease_class] = {'completed': 0, 'file': output_filename, 'needs_more': True}
        else:
            progress[disease_class] = {'completed': 0, 'file': output_filename, 'needs_more': True}
    
    return progress

# ============================================
# FONCTION POUR COMPLÉTER UNE CLASSE
# ============================================
def complete_class_conversations(disease_class, pdf_filename, existing_count):
    """Génère les conversations manquantes pour une classe"""
    remaining = NUM_CONVERSATIONS_PER_CLASS - existing_count
    
    if remaining <= 0:
        print(f"  ✅ Déjà complet ({existing_count} conversations)")
        return []
    
    print(f"  🔄 Génère {remaining} conversations supplémentaires...")
    
    pdf_path = docs_folder / pdf_filename
    
    if not pdf_path.exists():
        print(f"  ❌ PDF introuvable")
        return []
    
    protocol_text = extract_text_from_pdf(pdf_path)
    
    # Génère les conversations manquantes
    new_conversations = create_medical_conversations(
        protocol_text, 
        disease_class, 
        remaining
    )
    
    return new_conversations

# ============================================
# AFFICHAGE DE L'ÉTAT ACTUEL
# ============================================
print("="*70)
print("🔍 VÉRIFICATION DE L'ÉTAT ACTUEL")
print("="*70)

progress = get_progress_status()

for disease_class, status in progress.items():
    completed = status['completed']
    percentage = (completed / NUM_CONVERSATIONS_PER_CLASS) * 100
    
    if status['needs_more']:
        print(f"📊 {disease_class}: {completed}/{NUM_CONVERSATIONS_PER_CLASS} ({percentage:.1f}%) - ⚠️ INCOMPLET")
    else:
        print(f"✅ {disease_class}: {completed}/{NUM_CONVERSATIONS_PER_CLASS} ({percentage:.1f}%) - COMPLET")

print("="*70)
print("\n🚀 REPRISE DE LA GÉNÉRATION\n")

# ============================================
# REPRISE DE LA GÉNÉRATION
# ============================================
for disease_class, pdf_filename in classes_config.items():
    status = progress[disease_class]
    
    print(f"\n[{disease_class}]")
    print(f"État: {status['completed']}/{NUM_CONVERSATIONS_PER_CLASS} conversations")
    
    if not status['needs_more']:
        print("  ✅ Déjà complet - IGNORÉ")
        continue
    
    # Charge les conversations existantes
    safe_filename = disease_class.lower().replace(' ', '_').replace('é', 'e').replace('è', 'e').replace('î', 'i')
    output_filename = f"{safe_filename}_conversations.json"
    output_path = output_folder / output_filename
    
    existing_conversations = []
    if output_path.exists():
        try:
            with open(output_path, 'r', encoding='utf-8') as f:
                existing_conversations = json.load(f)
            print(f"  📂 {len(existing_conversations)} conversations existantes chargées")
        except:
            print("  ⚠️ Erreur lecture fichier existant - recommence à 0")
            existing_conversations = []
    
    # Génère les conversations manquantes
    new_conversations = complete_class_conversations(
        disease_class, 
        pdf_filename, 
        len(existing_conversations)
    )
    
    if len(new_conversations) > 0:
        # Combine avec les existantes
        all_conversations = existing_conversations + new_conversations
        
        # Renumérotation des IDs
        base_name = disease_class.lower().replace(' ', '_').replace('é', 'e').replace('è', 'e').replace('î', 'i')
        for idx, conv in enumerate(all_conversations, 1):
            conv['image_id'] = f"{base_name}_{idx:03d}"
            conv['disease_class'] = disease_class
        
        # Sauvegarde
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(all_conversations, f, ensure_ascii=False, indent=2)
        
        print(f"  💾 SAUVEGARDE: {len(all_conversations)} conversations totales -> {output_filename}")
        print(f"     ({len(existing_conversations)} anciennes + {len(new_conversations)} nouvelles)")
    else:
        print(f"  ❌ Aucune nouvelle conversation générée")
    
    print("-"*70)

# ============================================
# RÉSUMÉ FINAL
# ============================================
print("\n" + "="*70)
print("📊 RÉSUMÉ FINAL")
print("="*70)

final_progress = get_progress_status()
total_conversations = sum(status['completed'] for status in final_progress.values())
total_expected = NUM_CONVERSATIONS_PER_CLASS * len(classes_config)

for disease_class, status in final_progress.items():
    completed = status['completed']
    percentage = (completed / NUM_CONVERSATIONS_PER_CLASS) * 100
    icon = "✅" if not status['needs_more'] else "⚠️"
    print(f"{icon} {disease_class}: {completed}/{NUM_CONVERSATIONS_PER_CLASS} ({percentage:.1f}%)")

print("="*70)
print(f"🎯 TOTAL: {total_conversations}/{total_expected} conversations")
print(f"📁 Dossier: {output_folder.absolute()}")
print("="*70)

🔍 VÉRIFICATION DE L'ÉTAT ACTUEL
📊 Urticaria Hives: 25/150 (16.7%) - ⚠️ INCOMPLET
📊 Eczema: 0/150 (0.0%) - ⚠️ INCOMPLET
📊 Atopic Dermatitis: 0/150 (0.0%) - ⚠️ INCOMPLET
📊 drug reactions: 0/150 (0.0%) - ⚠️ INCOMPLET
📊 Dermatophytosis: 0/150 (0.0%) - ⚠️ INCOMPLET
📊 Gale: 0/150 (0.0%) - ⚠️ INCOMPLET
📊 Impétigo: 0/150 (0.0%) - ⚠️ INCOMPLET
📊 Maladie cutanée auto-immune: 0/150 (0.0%) - ⚠️ INCOMPLET

🚀 REPRISE DE LA GÉNÉRATION


[Urticaria Hives]
État: 25/150 conversations
  📂 25 conversations existantes chargées
  🔄 Génère 125 conversations supplémentaires...
  Batch 1/62... ❌ Parse échec
  Batch 2/62... ✅ 2
  Batch 3/62... ✅ 2
  Batch 4/62... ✅ 2
  Batch 5/62... ✅ 2
  Batch 6/62... ✅ 2
  Batch 7/62... ✅ 2
  Batch 8/62... ❌ Rejeté
  Batch 9/62... ✅ 2
  Batch 10/62... ✅ 1
  Batch 11/62... ✅ 1
  Batch 12/62... ❌ Rejeté
  Batch 13/62... ✅ 2
  Batch 14/62... ✅ 2
  Batch 15/62... ✅ 2
  Batch 16/62... ✅ 2
  Batch 17/62... ✅ 2
  Batch 18/62... ✅ 2
  Batch 19/62... ✅ 2
  Batch 20/62... ✅ 2
  Batch 2

KeyboardInterrupt: 